In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sicelib.hammers import get_region_labels, SIM_SUBSET
from collections import namedtuple
import re
import os
import scipy
import networkx
import itertools
import nilearn
import nibabel
from scipy import ndimage as ndi

Load and normalise the structural connectivity matrices
=======================================================

The matrices are represented as an (n×m×m) array, where n is the number of subjects and m is the number of regions.  We normalise by dividing each element by the corresponding row plus column sums, and take a submatrix by deleting those rows which are not in our subset of 62 regions.

In [3]:
structural_connectivities = load("../classes/all/structural_connectivity.npy")
for sc in structural_connectivities:
    region_sums = np.sum(sc, axis=0)
    norm_matrix = region_sums[:, None] + region_sums
    norm_matrix[norm_matrix == 0] = 1
    sc /= norm_matrix
structural_connectivities = structural_connectivities[:, SIM_SUBSET, :][:, :, SIM_SUBSET]

A helper function to turn one structural connectivity matrix into a list of thresholded matrices, such that the first element is a network without structural connections, the second contains only the strongest structural connection, etc.

In [4]:
def threshold_matrix(matrix):
    thresholds = unique(matrix[triu_indices_from(matrix, 1)].flatten())
    thresholds[::-1].sort()
    thresholded_matrices = []
    for threshold in thresholds:
        m = matrix.copy()
        m[m < threshold] = 0
        thresholded_matrices.append(m)
    return thresholded_matrices

Find the metabolic connectivity matrices
========================================

Metabolic connectivity is estimated for the entire population as a whole.  We have obtained it for the actual data as well as 100 bootstrapped samples.  Each of these metabolic networks is represented as an (k×m×m) array, where m is the number of regions again, and k is the sparsity level (such that the kth matrix contains only the k strongest metabolic connections).

In [5]:
metabolic_dir = "results"
filename_re = re.compile(r"all_zero_simhammers(?:_bootstrap(\d|\d\d|100))?\.pkl$")
metabolic_files = []
for filename in os.listdir(metabolic_dir):
    mt = filename_re.match(filename)
    if mt:
        bootstrap_seed = None
        path = os.path.join(metabolic_dir, filename)
        if mt.group(1) is not None:
            bootstrap_seed = int(mt.group(1))
            metabolic_files.append((path, bootstrap_seed))
        else:
            metabolic_files.insert(0, (path, None))

This flag specifies whether or not negative correlations should be included in the analysis.

In [6]:
negative_metabolic_connections = False

Network similarity analysis
===========================

This follows the procedure by Gong et al., Convergence and divergence of thickness correlations with diffusion connections across the human cerebral cortex, NeuroImage 59:1239-1248, 2012.

In [ ]:
GongSimilarity = namedtuple("GongSimilarity", ["narcs", "similarity", "convergence"])

def compute_gong_similarity(structural_matrix, metabolic_matrices):
    structural_matrices = threshold_matrix(structural_matrix)
    possible_structural_narcs = [count_nonzero(sm[triu_indices_from(sm)]) for sm in structural_matrices]
    possible_metabolic_narcs = [count_nonzero(mm[triu_indices_from(mm)]) for mm in metabolic_matrices]
    narcs = intersect1d(possible_structural_narcs, possible_metabolic_narcs)
    narcs = setdiff1d(narcs, [0])

    accs = zeros(len(narcs))
    convs = zeros(len(narcs))

    for i, narc in enumerate(narcs):
        struct_index = possible_structural_narcs.index(narc)
        metabolic_index = possible_metabolic_narcs.index(narc)

        s = structural_matrices[struct_index] != 0
        s = s[triu_indices_from(s, 1)].flatten()

        m = metabolic_matrices[metabolic_index] != 0
        m = m[triu_indices_from(m, 1)].flatten()

        fp = count_nonzero(m & ~s)
        tn = count_nonzero(~m & ~s)

        tp = count_nonzero(m & s)
        fn = count_nonzero(~m & s)

        accs[i] = (tn + tp) / (tn + fn + fp + tp)
        convs[i] = count_nonzero(s & m) / narc
        
    return GongSimilarity(narcs=narcs, similarity=accs, convergence=convs)

Calculate the similarity scores on the real data and the bootstrap samples.

In [ ]:
narcs = arange(450) + 1
data_similarities = zeros((len(metabolic_files), len(narcs)))
data_convergences = zeros((len(metabolic_files), len(narcs)))

for i, (filename, bootstrap_seed) in enumerate(metabolic_files):
    if bootstrap_seed is not None:
        index_sequence = np.random.RandomState(bootstrap_seed). \
            choice(len(structural_connectivities),
                   size=len(structural_connectivities),
                   replace=True)
    else:
        index_sequence = arange(len(structural_connectivities))
    structural_matrix = np.mean(structural_connectivities[index_sequence], axis=0)
    metabolic_data = load(filename)
    if negative_metabolic_connections:
        metabolic_matrices = array(metabolic_data["prec"]) != 0
    else:
        metabolic_matrices = array(metabolic_data["prec"]) < 0
    gong_similarity = compute_gong_similarity(structural_matrix,
                                              metabolic_matrices)
    data_similarities[i] = scipy.interp(narcs, gong_similarity.narcs,
                                        gong_similarity.similarity)
    data_convergences[i] = scipy.interp(narcs, gong_similarity.narcs,
                                        gong_similarity.convergence)

Generate a set of independently randomly generated metabolic and structural networks to establish reference scores.

In [ ]:
n_chance_runs = 100
chance_similarities = zeros((n_chance_runs, len(narcs)))
chance_convergences = zeros((n_chance_runs, len(narcs)))
max_narcs = narcs[-1]
n_regions = len(SIM_SUBSET)

for i in range(n_chance_runs):
    triu_size = n_regions * (n_regions - 1) // 2
    x = concatenate([arange(max_narcs) + 1, zeros(triu_size - max_narcs)])
    
    shuffle(x)
    chance_metabolic = zeros((n_regions, n_regions))
    chance_metabolic[triu_indices_from(chance_metabolic, 1)] = x
    chance_metabolic += chance_metabolic.T

    shuffle(x)
    chance_structural = zeros((n_regions, n_regions))
    chance_structural[triu_indices_from(chance_structural, 1)] = x
    chance_structural += chance_structural.T
    
    gong_similarity = compute_gong_similarity(chance_structural,
                                              threshold_matrix(chance_metabolic))
    chance_similarities[i] = gong_similarity.similarity
    chance_convergences[i] = gong_similarity.convergence

Plot the whole thing.

In [ ]:
figsize(10, 5)
subplot(1, 2, 1)
bs_mean_curve, = plot(narcs, mean(data_similarities, axis=0), "b")
plot(narcs, mean(data_similarities, axis=0) + std(data_similarities, axis=0), "b--")
plot(narcs, mean(data_similarities, axis=0) - std(data_similarities, axis=0), "b--")
real_curve, = plot(narcs, data_similarities[0], "r")
chance_curve, = plot(narcs, mean(chance_similarities, axis=0), "k")
plot(narcs, mean(chance_similarities, axis=0) + std(chance_similarities, axis=0), "k--")
plot(narcs, mean(chance_similarities, axis=0) - std(chance_similarities, axis=0), "k--")
xlim(50, 400)
ylim(0.6, 1)
xlabel("Number of connections", fontsize=12)
ylabel("Similarity metric", fontsize=12)
legend([real_curve, bs_mean_curve, chance_curve],
       ["real network", "bootstrapped real networks (μ ± σ)", "random networks (μ ± σ)"],
       loc="lower left")
grid()

subplot(1, 2, 2)
plot(narcs, mean(data_convergences, axis=0), "b")
plot(narcs, mean(data_convergences, axis=0) + std(data_convergences, axis=0), "b--")
plot(narcs, mean(data_convergences, axis=0) - std(data_convergences, axis=0), "b--")
plot(narcs, data_convergences[0], "r")
plot(narcs, mean(chance_convergences, axis=0), "k")
plot(narcs, mean(chance_convergences, axis=0) + std(chance_convergences, axis=0), "k--")
plot(narcs, mean(chance_convergences, axis=0) - std(chance_convergences, axis=0), "k--")
xlim(50, 400)
ylim(0, 0.8)
xlabel("Number of connections", fontsize=12)
ylabel("Convergence ratio", fontsize=12)
grid()

suptitle("Metabolic and structural connectivity networks", fontsize=14)
tight_layout(rect=[0, 0, 1, 0.95])

Efficiency analysis
===================

Compare some structural properties of the graphs.

In [ ]:
def distance_matrix(g):
    dist_dict = networkx.all_pairs_shortest_path_length(g)
    dist = numpy.full((len(g), len(g)), inf)
    for i, v in enumerate(g):
        if v not in dist_dict: continue
        for j, w in enumerate(g):
            if w not in dist_dict[v]: continue
            dist[i, j] = dist_dict[v][w]
    return dist

In [ ]:
NetworkMeasures = namedtuple("NetworkMeasures", ["narcs", "global_efficiencies", "local_efficiencies"])

def compute_network_measures(matrices):
    if type(matrices) is ndarray and matrices.ndim == 2:
        matrices = threshold_matrix(matrices)

    local_efficiencies = []
    global_efficiencies = []
    narcs = []

    for matrix in matrices:
        if type(matrix) is not networkx.Graph:
            g = networkx.Graph(matrix != 0)
        else:
            g = matrix
        dist_dict = networkx.all_pairs_shortest_path_length(g)
        dist = full((len(g), len(g)), inf)
        for i in dist_dict.keys():
            for j in dist_dict[i].keys():
                dist[i, j] = dist_dict[i][j]

        narcs.append(g.size())
        global_efficiencies.append((1 / dist[triu_indices_from(dist, 1)]).mean())

        le = []
        for v in g:
            g_sub = g.subgraph(g[v])
            if len(g_sub) <= 1:
                le.append(0)
                continue
            sub_dist_dict = networkx.all_pairs_shortest_path_length(g_sub)
            sub_dist = full((len(g_sub), len(g_sub)), inf)
            vertex_map = {w: i for i, w in enumerate(g_sub.nodes())}
            for i in sub_dist_dict.keys():
                for j in sub_dist_dict[i].keys():
                    sub_dist[vertex_map[i], vertex_map[j]] = sub_dist_dict[i][j]
            le.append((1 / sub_dist[triu_indices_from(sub_dist, 1)]).mean())

        local_efficiencies.append(mean(le))
        
    order = argsort(narcs)
        
    return NetworkMeasures(narcs=np.array(narcs)[order],
                           global_efficiencies=np.array(global_efficiencies)[order],
                           local_efficiencies=np.array(local_efficiencies)[order])

Individually compute local and global efficiency scores for the structural and metabolic networks, for the real data and the bootstrapped samples.  This takes a good while; you may want to put a “break” at the end of the loop to skip the bootstrapped data.

In [ ]:
narcs = arange(450) + 1
data_met_local_efficiencies = zeros((len(metabolic_files), len(narcs)))
data_met_global_efficiencies = zeros((len(metabolic_files), len(narcs)))
data_struct_local_efficiencies = zeros((len(metabolic_files), len(narcs)))
data_struct_global_efficiencies = zeros((len(metabolic_files), len(narcs)))

for i, (filename, bootstrap_seed) in enumerate(metabolic_files):
    if bootstrap_seed is not None:
        index_sequence = np.random.RandomState(bootstrap_seed). \
            choice(len(structural_connectivities),
                   size=len(structural_connectivities),
                   replace=True)
    else:
        index_sequence = arange(len(structural_connectivities))
    structural_matrix = np.mean(structural_connectivities[index_sequence], axis=0)
    metabolic_data = load(filename)
    if negative_metabolic_connections:
        metabolic_matrices = array(metabolic_data["prec"]) != 0
    else:
        metabolic_matrices = array(metabolic_data["prec"]) < 0
    data_met_network_measures = compute_network_measures(metabolic_matrices)
    data_met_local_efficiencies[i] = scipy.interp(narcs, data_met_network_measures.narcs,
                                                  data_met_network_measures.local_efficiencies)
    data_met_global_efficiencies[i] = scipy.interp(narcs, data_met_network_measures.narcs,
                                                   data_met_network_measures.global_efficiencies)
    data_struct_network_measures = compute_network_measures(structural_matrix)
    data_struct_local_efficiencies[i] = scipy.interp(narcs, data_struct_network_measures.narcs,
                                                     data_struct_network_measures.local_efficiencies)
    data_struct_global_efficiencies[i] = scipy.interp(narcs, data_struct_network_measures.narcs,
                                                      data_struct_network_measures.global_efficiencies)

The following function creates a random, but comparable graph to an input, in that they have the same degree distribution.

In [ ]:
def random_rewire(graph):
    graph = graph.copy()
    for iteration in range(graph.size()):
        e = graph.edges()
        while True:
            i1, i2 = np.random.choice(arange(len(e)), size=2, replace=False)
            e1, e2 = e[i1], e[i2]
            new_e1 = e1[0], e2[1]
            new_e2 = e1[1], e2[0]
            if new_e1[0] != new_e1[1] and new_e2[0] != new_e2[1] and \
                    not graph.has_edge(*new_e1) and not graph.has_edge(*new_e2):
                break
        graph.remove_edge(*e1)
        graph.remove_edge(*e2)
        graph.add_edge(*new_e1)
        graph.add_edge(*new_e2)
    return graph

Run the computation again for some of these randomly rewired graphs.

In [ ]:
n_chance_runs = 100
chance_met_local_efficiencies = zeros((n_chance_runs, len(narcs)))
chance_met_global_efficiencies = zeros((n_chance_runs, len(narcs)))
chance_struct_local_efficiencies = zeros((n_chance_runs, len(narcs)))
chance_struct_global_efficiencies = zeros((n_chance_runs, len(narcs)))

# the randomly rewired graphs are modelled after the real (non-bootstrapped) data
metabolic_data = load(metabolic_files[0][0])
if negative_metabolic_connections:
    metabolic_matrices = array(metabolic_data["prec"]) != 0
else:
    metabolic_matrices = array(metabolic_data["prec"]) < 0
for m in metabolic_matrices:
    fill_diagonal(m, False)
metabolic_graphs = [networkx.Graph(m) for m in metabolic_matrices if count_nonzero(m) // 2 >= 10]

structural_matrix = np.mean(structural_connectivities, axis=0)
structural_matrices = threshold_matrix(structural_matrix)
structural_graphs = [networkx.Graph(m != 0) for m in structural_matrices if count_nonzero(m != 0) // 2 >= 10]

for i in range(n_chance_runs):
    print("{}/{}".format(i + 1, n_chance_runs))
    chance_met_network_measures = compute_network_measures([random_rewire(g) for g in metabolic_graphs])
    chance_met_local_efficiencies[i] = scipy.interp(narcs, chance_met_network_measures.narcs,
                                                    chance_met_network_measures.local_efficiencies)
    chance_met_global_efficiencies[i] = scipy.interp(narcs, chance_met_network_measures.narcs,
                                                     chance_met_network_measures.global_efficiencies)
    chance_struct_network_measures = compute_network_measures([random_rewire(g) for g in structural_graphs])
    chance_struct_local_efficiencies[i] = scipy.interp(narcs, chance_struct_network_measures.narcs,
                                                       chance_struct_network_measures.local_efficiencies)
    chance_struct_global_efficiencies[i] = scipy.interp(narcs, chance_struct_network_measures.narcs,
                                                        chance_struct_network_measures.global_efficiencies)

And plot the result.

In [ ]:
figsize(10, 10)
titles = ["Global efficiency of the metabolic network",
          "Global efficiency of the structural network",
          "Local efficiency of the metabolic network",
          "Local efficiency of the structural network"]
for i, (de, ce) in enumerate([(data_met_global_efficiencies, chance_met_global_efficiencies),
                              (data_struct_global_efficiencies, chance_struct_global_efficiencies),
                              (data_met_local_efficiencies, chance_met_local_efficiencies),
                              (data_struct_local_efficiencies, chance_struct_local_efficiencies)]):
    subplot(2, 2, i+1)
    title(titles[i])
    bs_mean_curve, = plot(narcs, mean(de, axis=0), "b")
    plot(narcs, mean(de, axis=0) + std(de, axis=0), "b--")
    plot(narcs, mean(de, axis=0) - std(de, axis=0), "b--")
    real_curve, = plot(narcs, de[0], "r")
    chance_curve, = plot(narcs, mean(ce, axis=0), "k")
    plot(narcs, mean(ce, axis=0) + std(ce, axis=0), "k--")
    plot(narcs, mean(ce, axis=0) - std(ce, axis=0), "k--")
    ylim(0, 0.8)
    if i == 1:
        legend([real_curve, bs_mean_curve, chance_curve],
               ["real network", "bootstrapped networks (μ ± σ)", "randomly rewired networks (μ ± σ)"], loc="upper left")

Structural and metabolic hubs
=============================

Again following Gong et al., for both the structural and metabolic networks, we consider vertices with a regional efficiency which exceeds the mean plus one standard deviation to be hubs.

In [ ]:
def regional_efficiencies(matrix):
    g = networkx.Graph(matrix != 0)
    dist = distance_matrix(g)
    vertices = []
    regional_efficiencies = []
    for i, v in enumerate(g):
        vertices.append(v)
        local_dist = delete(delete(dist, i, axis=0), i, axis=1)
        regional_efficiencies.append(mean(1 / local_dist[triu_indices_from(local_dist, 1)]))
    regional_efficiencies = array(regional_efficiencies)
    vertices = array(vertices)
    regional_efficiencies -= mean(regional_efficiencies)
    regional_efficiencies /= std(regional_efficiencies)
    indices = argsort(regional_efficiencies)[::-1]
    return zip(vertices[indices], regional_efficiencies[indices])

Work with the full, non-bootstrapped dataset.

In [ ]:
metabolic_matrices = array(load(metabolic_files[0][0])["prec"])
if negative_metabolic_connections:
    metabolic_matrices = metabolic_matrices != 0
else:
    metabolic_matrices = metabolic_matrices < 0
structural_matrix = mean(structural_connectivities, axis=0)
structural_matrices = threshold_matrix(structural_matrix)

Find the structural and metabolic matrices with a fixed number of strongest connections.

In [ ]:
n_connections = 150
metabolic_index = argmin(np.abs(n_connections - array([count_nonzero(m) for m in metabolic_matrices]) // 2))
structural_index = argmin(np.abs(n_connections - array([count_nonzero(m) for m in structural_matrices]) // 2))

Identify the hubs.

In [ ]:
metabolic_hubs = set(region_labels[i] for i, eff in regional_efficiencies(metabolic_matrices[metabolic_index]) if eff > 1)
structural_hubs = set(region_labels[i] for i, eff in regional_efficiencies(structural_matrices[metabolic_index]) if eff > 1)

In [ ]:
print("\n".join(metabolic_hubs & structural_hubs))

In [ ]:
print("\n".join(metabolic_hubs.difference(structural_hubs)))

In [ ]:
print("\n".join(structural_hubs.difference(metabolic_hubs)))

Visualisation of individual networks
====================================

Define boolean matrices which are True for all connections between hemispheres (interhemispheric_mak), or between all homologous regional pairs (homologous_mask).

In [ ]:
region_labels = get_region_labels(SIM_SUBSET)
left_regions, right_regions = array([[l.endswith("_L"), l.endswith("_R")] for l in region_labels]).T

homologous_mask = zeros_like(structural_matrix, dtype=bool)
homologous_mask[left_regions, right_regions] = True
homologous_mask[right_regions, left_regions] = True

interhemispheric_mask = zeros_like(structural_matrix, dtype=bool)
for i, j in itertools.product(where(left_regions)[0], where(right_regions)[0]):
    interhemispheric_mask[i, j] = interhemispheric_mask[j, i] = True

Plot convergent and divergent connections as a matrix…

In [ ]:
figsize(12, 10)
convergence = 2 * metabolic_matrices[metabolic_index].astype(int) + (structural_matrices[structural_index] != 0).astype(int)
imshow(convergence, interpolation="none", cmap=cm.get_cmap("cubehelix_r", 4))
colorbar(ticks=[0, 1, 2, 3], format=FuncFormatter(lambda val, loc: {0: "not connected",
                                                                    1: "structural connection only",
                                                                    2: "metabolic connection only",
                                                                    3: "structural and metabolic connections"}[val]),
         shrink=0.5)
clim(-0.5, 3.5)
xticks(arange(len(region_labels)), region_labels, rotation=90);
yticks(arange(len(region_labels)), region_labels);
grid();
tight_layout()

Plot the same information on a drawing of a brain.

In [ ]:
atlas = nibabel.load("../../data/dti_ftlad/atlases/hammers/Hammers_mith_atlas_n30r83_SPM5.nii.gz")
atlas_data = atlas.get_data()

In [ ]:
centers_vox = ndi.center_of_mass(ones_like(atlas_data), atlas_data, SIM_SUBSET + 1)
centers_vox_hom = hstack([array(centers_vox), ones((len(centers_vox), 1))])
centers_mm_hom = atlas.affine.dot(centers_vox_hom.T).T
centers_mm = centers_mm_hom[:, 0:3]

In [ ]:
fig = figure(figsize=(9, 3))
nilearn.plotting.plot_connectome(convergence, centers_mm, edge_cmap="cubehelix_r", edge_vmin=0, edge_vmax=3, figure=fig)

Frequency of interhemispheric connections
=========================================

In [ ]:
metabolic_narcs = [count_nonzero(mm) // 2 for mm in metabolic_matrices]
structural_narcs = [count_nonzero(sm) // 2 for sm in structural_matrices]
narcs = intersect1d(metabolic_narcs, structural_narcs)

In [ ]:
interhemispheric = zeros((2, len(narcs)))
homologous = zeros((2, len(narcs)))

In [ ]:
for i, narc in enumerate(narcs):
    mm = metabolic_matrices[metabolic_narcs.index(narc)]
    sm = structural_matrices[structural_narcs.index(narc)]
    interhemispheric[:, i] = [count_nonzero((m != 0) & interhemispheric_mask) / count_nonzero(m)
                              for m in [mm, sm]]
    homologous[:, i] = [count_nonzero((m != 0) & homologous_mask) / count_nonzero(m)
                        for m in [mm, sm]]

In [ ]:
figure(figsize=(10, 4))
subplot(121)
plot(interhemispheric.T)
title("Prevalence of interhemispheric connections")
legend(["metabolic network", "structural network"])
subplot(122)
plot(homologous.T)
title("Prevalence of homologous connections")
tight_layout()

Distances in the networks
=========================

In [ ]:
gm_probability = nilearn.image.index_img(nibabel.load("/home/michael/software/spm12/tpm/TPM.nii"), 0)
gm_probability = nilearn.image.resample_img(gm_probability, atlas.affine, atlas.shape)
atlas_data[gm_probability.get_data() < 0.8] = 0